In [1]:
!pip install langchain pinecone-client openai tiktoken langchain_openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.0/817.0 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.0/211.0 kB 25.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 30.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 26.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 67.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 246.4/246.4 kB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.2/62.2 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 20.6 MB/s eta 0:00:00


In [2]:
import os
# import getpass
os.environ["OPENAI_API_KEY"] = ""       #@param {type:"string"}
os.environ["PINECONE_API_KEY"] = ""      #@param {type:"string"}

In [14]:
from langchain.memory import ConversationSummaryMemory, ChatMessageHistory
from langchain_openai import OpenAI
from langchain.chains import ConversationChain

from langchain.vectorstores import Pinecone
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains.question_answering import load_qa_chain
from langchain.document_loaders import TextLoader
from langchain.schema import SystemMessage
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.output_parsers import PydanticOutputParser
from langchain.pydantic_v1 import BaseModel, Field, validator
import langchain, pinecone

from pinecone import Pinecone as pc
from pinecone import PodSpec


In [35]:
store = {}

In [36]:

#+================================== prompt ==================================+
prompt = '''You will first make a greeting to me and answer my question. All reponses should be based on my information: "{info}".

If there is no name in the information, you should say: Hi, {id}!

If there is a name in the information, you should greet me with my name.

Here is my question: {question}

Output:'''

#+=============================== calling LLM ===============================+
llm = OpenAI(temperature=0)

_prompt = PromptTemplate(template =  prompt, input_variables = ["info", "question"])

while True:
  print("="*10, f"New session", "="*10)

  id = input(f"Please enter your id: ")          # id for retreving information
  if id.lower() in ['q','quit','exit']:
    break

  if id not in store:                    # record info
    info = input(f"Please tell me something about you, this helps me improve the answer: ")
    store[id] = info
  else:
    info = store[id]                    # using previous info

  while True:
    memory = ConversationSummaryMemory(llm=OpenAI())
    conversation_with_summary = ConversationChain(
      llm=llm,
      memory=memory,
      verbose=False
    )
    user_input = input(f"{id} > ")

    if user_input.lower() in ["q", "quit", "exit"]:    # quit the current conversation
      new_info = memory.predict_new_summary(memory.chat_memory.messages, info)
      store[id] = new_info
      break

    _input = _prompt.format_prompt(id = id, info = info, question = user_input).to_string()

    result = conversation_with_summary.predict(input = _input)
    print(result)

  if user_input.lower() == "exit":               # quit the QA system
    print("="*10, f"Thank you for using!", "="*10)
    break

========== New session ==========
Please enter your id: vyaas
Please tell me something about you, this helps me improve the answer: I am a non vegetarian.
vyaas > What should I eat for lunch?
 Hi, vyaas! As a non vegetarian, you have many options for lunch. You could have a chicken sandwich, a beef stir fry, or a tuna salad. Do any of those sound appealing to you?
vyaas > q
========== New session ==========
Please enter your id: messi
Please tell me something about you, this helps me improve the answer: I am a vegetarian.
messi > What should I eat for lunch?
 Hi, messi! As a vegetarian, you have many options for lunch. You could have a salad with fresh greens, vegetables, and a protein source like tofu or beans. Another option could be a veggie wrap with hummus, avocado, and your choice of veggies. You could also try a quinoa and vegetable stir-fry or a lentil soup. The possibilities are endless! Is there a specific type of cuisine you are in the mood for?
messi > q
========== New sess

In [37]:
store

{'vyaas': '\nThe human asks if the AI has any dietary restrictions. The AI responds that it is a non-vegetarian. The human then asks why the AI is not a vegetarian. The AI explains that it does not require sustenance in the traditional sense and does not have the same ethical considerations as humans.',
 'messi': '\nThe human asks the AI if it has any dietary restrictions. The AI responds that it is a vegetarian.'}

## backup code

In [42]:
#+============================= text processing =============================+
loader = TextLoader("./example.txt")
documents = loader.load()
#splitting the text into smaller chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(documents)


#+============================== embedding text =============================+
llm= OpenAI()
embeddings = OpenAIEmbeddings()

pc_ = pc(api_key=os.environ.get("PINECONE_API_KEY"))

index_name = "example-index"

if index_name not in pc_.list_indexes().names():
    pc_.create_index(name=index_name, metric="cosine", dimension=1536, spec=PodSpec(environment="gcp-starter") )
    docsearch = Pinecone.from_documents(texts, embeddings, index_name=index_name)
else:
    docsearch = Pinecone.from_existing_index(index_name, embeddings)

chain = load_qa_chain(llm, chain_type="stuff")

#+================================= QA Bot ==================================+
prompts=["who is meta?","what is a toolformer?","what is an llm?"]

for i in range(3):
  if not prompts:
    question = input(f"Question {i+1}> ")
  else:
    question=prompts[i]
    print(f"Question {i+1}:{question}")
  docs = docsearch.similarity_search(question)
  respond = chain.run(input_documents=docs, question=question)
  print(f"Response {i+1}:{respond}")





Question 1:who is meta?
Response 1:

Meta is a company or organization that published the paper discussing a solution for LLMs using external tools through APIs.
Question 2:what is a toolformer?
Response 2:

A toolformer is a framework developed by Meta AI that allows large language models to use external tools through simple APIs, combining the benefits of both LLMs and traditional software interfaces. It integrates a range of tools, including a calculator, Q&A system, search engines, translation system, and calendar.
Question 3:what is an llm?
Response 3:
LLM stands for large language models. These are artificial intelligence systems that are trained on large amounts of text data and can generate natural-sounding text. However, they may still have issues with accuracy and generating false information.


# Supplemental Q2

In [43]:
#We can either use counter to allow maximum questions to break a loop or ask user to enter close
flag=True
print("Enter close to exit\n\n")
while True:
  question=input("User:\n")
  if question.lower()=="close":
    break
  if flag:
    first_q=question
    print("Response: Before I answer your question, may I know your name?\n")
    question=input("User:")
    name=llm(f"Extract the name from the sentence:{question}")[2:]
    response=chain.run(input_documents=docs, question=first_q)
    print(f"Response: Hello, {name}. Hope you are having a good day so far. {response}\n")
    flag=False
  else:
    response=chain.run(input_documents=docs, question=question)
    print(f"Response: Hello, {name}. Hope you are having a good day so far. {response}\n")

Enter close to exit


User:
what is a toolformer?
Response: Before I answer your question, may I know your name?

User:Vyaas


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


Response: Hello, Vyaas. Hope you are having a good day so far. 
A toolformer is a framework developed by Meta AI that allows large language models (LLMs) to use external tools through simple APIs, combining the strengths of both approaches. It integrates various tools such as a calculator, Q&A system, search engines, translation system, and calendar to help address the limitations of LLMs in generating factual and accurate information.

User:
who is meta?
Response: Hello, Vyaas. Hope you are having a good day so far. 

Meta AI is the company that presented the solution for LLMs to use external tools via simple APIs, called Toolformer. They also integrate a range of tools, including a calculator, a Q&A system, two search engines, a translation system, and a calendar.

User:
close


Optional: Multiple users attempt

In [54]:
llm=OpenAI()
print("Enter \"close\" to exit as a user\nEnter \"quit\" to stop running the cell\n\n")
names=[]
while True:
  flag=True
  while True:
    question=input("User:")

    #Checks whether the user wants to exit or quit
    if question.lower()=="close" or question.lower()=="quit":
      #Breaks the inner loop but cell may keep running
      print("\nUser Exiting")
      print("="*20,"\n")
      break

    #Checks if this is the first question by the user in the current session
    if flag:
      first_q=question
      print("Response: Before I answer your question, may I know your name?\n")
      question=input("User:")
      name=llm(f"Extract the name from the sentence:{question}")[2:]
      response=chain.run(input_documents=docs, question=first_q)
      #Prints different outputs if this is the first time the QA bot is meeting this person
      if name in names:
        print(f"Response: Hello, {name}. It is nice to see you again. Hope you are having a good day so far. {response}\n")
      else:
        names.append(name)
        print(f"Response: Hello, {name}. It is nice to meet you. Hope you are having a good day so far. {response}\n")

      flag=False

    #Response if it is not the first question by the user in the current session
    else:
      response=chain.run(input_documents=docs, question=question)
      print(f"Response: Hello, {name}. Hope you are having a good day so far. {response}\n")
  if question.lower()=="quit":
    #Breaks the outer loop and the cell stops running
    print(""*20)
    print("Over and out!")
    print(""*20)
    break

Enter "close" to exit as a user
Enter "quit" to stop running the cell


User:what is a toolformer?
Response: Before I answer your question, may I know your name?

User:My name is Vyaas
Response: Hello, Vyaas. It is nice to meet you. Hope you are having a good day so far. 
A toolformer is a framework developed by Meta AI that integrates a range of external tools, such as a calculator, Q&A system, search engines, translation system, and a calendar, to enhance the capabilities of large language models (LLMs). This approach aims to combine the benefits of LLMs with traditional software interfaces to solve crucial problems in natural language processing.

User:close

User Exiting

User:what is a toolformer?
Response: Before I answer your question, may I know your name?

User:The name is Ronaldo.
Response: Hello, Ronaldo. It is nice to meet you. Hope you are having a good day so far.  A toolformer is a framework developed by Meta AI that allows large language models to use external tools thr